In [1]:
# widen jupyter notebook window
from IPython.core.display import display, HTML
display(HTML("<style>.container {width:95% !important; }</style>"))

In [ ]:
# import libraries
import sys
import cv2
import numpy as np
import skimage.draw


this is just a short script to figure out what the absolute length of pixels are so you can convert the y-axes of plots from pixels to millimeters or whatever

In [3]:
## prep stuff
## find slash type of operating system

if sys.platform == 'linux':
    slash_type = '/'
    print('Autodetected operating system: Linux. Using "/" for directory slashes')
elif sys.platform == 'win32':
    slash_type = '\\'
    print(f'Autodetected operating system: Windows. Using "{slash_type}{slash_type}" for directory slashes')
elif sys.platform == 'darwin':
    slash_type = '/'
    print("What computer are you running this on? I haven't tested it on OSX or anything except windows and ubuntu.")
    print('Autodetected operating system: OSX. Using "/" for directory slashes')

Autodetected operating system: Linux. Using "/" for directory slashes


In [4]:
### Define LOAD DIRECTORY and FILENAME

dir_vid = r'/media/rich/bigSSD RH/aniqa_tmp'
fileName_vid = 'fc2_save_2020-11-13-173638-0000.avi'
path_vid = f'{dir_vid}{slash_type}{fileName_vid}'


In [10]:
### Define SAVE DIRECTORY: for camera_distancePerPixel.npy file
dir_save = r'/media/rich/bigSSD RH/aniqa_tmp'
name_save = 'camera_distancePerPixel'
path_save = f'{dir_save}{slash_type}{name_save}.npy'

In [24]:
### Select POLYGON SUBFRAME for DISPLACEMENT Eignfaces
## This block of code will pop up a little GUI. Click around the
## region of the face that you want to include in the analysis.
## When you are done, press enter twice to accept and exit the GUI.

frameToSet = 1 # 1 indexed. Sets the frame number to use to make an image

video = cv2.VideoCapture(path_vid)
    
video.set(1,frameToSet)
ok, frame = video.read()
cv2.imshow('window', frame)
# cv2.waitKey(1);

## these colors aren't really important, they just allow the text and lines in the GUI to change color
RED = (0, 0, 255)
GREEN = (0, 255, 0)
BLUE = (255, 0, 0)
CYAN = (255, 255, 0)
MAGENTA = (255, 0, 255)
YELLOW = (0, 255, 255)
WHITE = (255, 255, 255)

colors = (WHITE , RED, GREEN, BLUE, CYAN, MAGENTA, YELLOW)

## The below block is adapted code. It makes a GUI, then allows a user to click to define the
## outline of the ROI to use. 'pts' are the clicked points.
pts = [] # prepare for appending. I'm using this global in functions like a pleb. please forgive
def draw(x):
    d = cv2.getTrackbarPos('thickness', 'window')
    d = 1 if d==0 else d
    i = cv2.getTrackbarPos('color', 'window')
    color = colors[i]
    cv2.polylines(frame, np.array([pts]), False, color, d)
    cv2.imshow('window', frame)
    text = f'color={color}, thickness={d}'
#     cv2.displayOverlay('window', text)


def mouse(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        if len(pts) < 2:
            pts.append((x, y))
            draw(0)

            
cv2.setMouseCallback('window', mouse)
cv2.createTrackbar('color', 'window', 0, 6, draw)
cv2.createTrackbar('thickness', 'window', 1, 10, draw)
cv2.putText(frame, f'Click on two points separated by a known distance', org=(10,20), fontFace=1, fontScale=1, color=(255,255,255), thickness=1)
cv2.putText(frame, f'then press enter', org=(10,35), fontFace=1, fontScale=1, color=(255,255,255), thickness=1)

draw(0)
cv2.waitKey(0)

cv2.destroyAllWindows()

distance_betweenPoints = np.linalg.norm(np.array(pts[1]) - np.array(pts[0]))


In [29]:
distance_userInput = input('Enter distance between the two clicked points (in centimeters):  ')
distance_perPixel = np.double((np.double(distance_userInput)*10) / distance_betweenPoints)
print(f'Distance per pixel in pixels:  {round(distance_betweenPoints,1)} pixels')
print(f'Distance per pixel in millimeters:  {round(distance_perPixel,4)} mm/pixel')

Enter distance between the two clicked points (in centimeters):  4
Distance per pixel in millimeters:  0.0587 mm/pixel


In [20]:
np.save(path_save , distance_perPixel)